In [1]:
import os


In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\MyData\\Projects-github\\NLP\\Text-Summarization-NLP'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [7]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-08-25 12:56:39,621: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-25 12:56:39,624: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-25 12:56:39,626: INFO: common: created directory at: artifacts]
[2023-08-25 12:56:39,629: INFO: common: created directory at: artifacts/data_transformation]


Downloading: 100%|██████████| 88.0/88.0 [00:00<00:00, 87.6kB/s]
Downloading: 100%|██████████| 1.09k/1.09k [00:00<00:00, 559kB/s]
Downloading: 100%|██████████| 1.82M/1.82M [00:00<00:00, 10.1MB/s]
Downloading: 100%|██████████| 65.0/65.0 [00:00<00:00, 32.5kB/s]


[2023-08-25 12:56:47,183: WARNING: fingerprint: Parameter 'function'=<function DataTransformation.convert_examples_to_features at 0x000001E9B9764048> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.]


100%|██████████| 1/1 [00:00<00:00,  1.98ba/s]
